<a href="https://colab.research.google.com/github/majaszymajda/PNoD_rozpoznawanie_obraz/blob/main/projekt_koncowa_wersja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wczytanie danych


## Zadanie 3
Należy sprawdzić jakość jednego z modeli z poprzednich zadań (Naive Bayes, K-NN, Regresja  Logistyczna) na zbiorzeFashionMnist. W tym celu należy wyuczyć model na danych treningowych i ocenić jego jakość na danych testowych w oparciu o poprawność klasyfikacji.W dalszej kolejności należy skonfrontować swój wynik z wynikiem otrzymanym w ramach http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/dla  odpowiadającejmetody i przeanalizować ewentualne różnice.
# Zadanie 4 
W tym podpunkcie można puścić wodzę fantazji i opracować własny algorytm, bądź algoryt-my do klasyfikacji zbioru FashionMnist. W ramach tego punktu możliwe jest korzystaniez gotowych bibliotek typuKeras, TensorFlow, pytorch, scikitoraz innych. Bardzo istotne jest, żeby opracowany model był uczony na danych treningowych, a oceniany na danych testowych - niedopuszczalne jest wykorzystanie danych testowych w procesie uczenia. W ra-mach danych treningowych można wyodrębnić zbiór walidacyjny na potrzeby selekcji modelu(kalibracji hiperparametrów). Do oceny opracowanej metody należy użyć miary poprawności klasyfikacji,  natomiast  można  badać  inne  miary  jakości,  np.F-Score.  Przez  opracowanie modelu rozumiemy tutaj zastosowanie takich technik augmentacji danych, ekstrakcji cech, doboru modelu i metody uczenia, aby maksymalizować jakość modelu na danych testowych.Wynik działania metody, bądź metod można skonfrontować z wynikami w sekcji Benchmarkna stronie na której udostępniony jest zbiórFasionMnist.

# Zadania do zrealizowania 
1. Przetworzenie danych
2. Zbudowanie modelu 
3. Trenowanie modelu
4. Karmienie Modelu
5. Ocena dokładności
6. Prognozowanie 
7. Weryfikacja na danych walidacyjnych
8. Wykorzystanie na danych testowych 

In [2]:
import os
import io
import gzip
import time
import gc
import numpy as np
import pandas as pd
import sklearn
import requests
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, SGDRegressor, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression


# Import plików z githuba

In [3]:
files = [
  "t10k-images-idx3-ubyte.gz",
  "t10k-labels-idx1-ubyte.gz",
  "train-images-idx3-ubyte.gz",
  "train-labels-idx1-ubyte.gz",
]

def download_and_save_locally(file):
    data = requests.get(f'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/{file}')
    if data.status_code != 200:
      print(f"Blad pobierania pliku {file}")

    with open(file, "wb") as f:
      f.write(data.content)

    print(f"Pobrano plik {file}")
    # unzipped_content = gzip.open(io.BytesIO(data))
   # f'df{file}' = pd.read_csv(unzipped_content, delimiter='\t')
 
    # df{file}.head()

for f in files:
    download_and_save_locally(f)

Pobrano plik t10k-images-idx3-ubyte.gz
Pobrano plik t10k-labels-idx1-ubyte.gz
Pobrano plik train-images-idx3-ubyte.gz
Pobrano plik train-labels-idx1-ubyte.gz


# Przetworzenie danych 

In [4]:
def load_mnist(kind):
    """Load MNIST data from `path`"""
    with gzip.open(f'{kind}-labels-idx1-ubyte.gz', 'rb') as lbpath:
        # lbpath = pd.read_csv(lbpath)
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                              offset=8)

    with gzip.open(f'{kind}-images-idx3-ubyte.gz', 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                              offset=16).reshape(len(labels), 784)

    return images, labels

In [5]:
X_train, y_train = load_mnist(kind='train')
X_test, y_test = load_mnist(kind='t10k')

print(X_train.shape)
print(X_test.shape)
print(y_train)
# klasy podane na gicie numerowane od 0 do 9 
class_clothes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)

(60000, 784)
(10000, 784)
[9 0 0 ... 3 0 5]


Próba wykorzystania algorytmu napisanego do metody kNN na zajęciach do listy 2

In [ ]:
def sort_train_labels_knn(Dist, y):
    
    return y[Dist.argsort(kind='mergesort')]

In [ ]:
def hamming_distance(X, X_train):
    Xrev = 1-X
    X = X.astype(int)
    X_trainrev = 1 - X_train
    X_trainint = X_train.astype(int)
    # print(Xrev @ X_trainint.transpose() + X @ X_trainrev.transpose())
    return Xrev @ X_trainint.transpose() + X @ X_trainrev.transpose()

In [ ]:

def model_selection_knn(Xval, Xtrain, yval, ytrain, k_values):
    
    errors = []
    srted_et = sort_train_labels_knn(hamming_distance(Xval, Xtrain), ytrain)
    best_k = 0
    min_er = 1
    for k in k_values:
        er = classification_error(p_y_x_knn(srted_et, k), yval)
        errors.append(er)
        if er < min_er:
            best_k = k
            min_er = er
    return min_er, best_k, errors

# Ekstrakcja cech
zamiast filtrów Haara używam randoma 



In [13]:
np.random.seed(0)
val_1_id = np.random.choice(60000, 10000, replace=False)
val_2_id = np.random.choice(list(set(range(60000)) - set(val_1_id)), 10000, replace=False)
train_id = list(set(range(60000))- set(val_2_id)- set(val_1_id))
test_id = np.random.choice(10000, 1, replace = False)

X_train_1 = X_train[train_id]
y_train_1 = y_train[train_id]

x_valid_1 = X_train[val_2_id]
y_valid_1 = y_train[val_2_id]

x_test_1 = X_test[test_id]
y_test_1 = y_test[test_id]



Jako metodę klasyfikacji służąca do budowy modelu wykorzystamy kNN. Dla 1, 5 i 9 sąsiadów. 

In [ ]:
# model.fit(x,y) - zajmuje się dopasowaniem modelu do danych 
# moja funkcja KNeighborsClassifier ma tylko jeden argument ponieważ algorytm nie ustawiony ma deafault auto - czyli wybierze najbardziej optymalna metode
# możliwe metody to: 
KNN_model_k_1 = KNeighborsClassifier(n_neighbors=1).fit(X_train_1, y_train_1)
confusion_matrix(KNN_model_k_1.predict(X_train_1), y_train_1)

In [10]:
# ocena dokładności 
KNN_model_k_1_accuracy = (KNN_model_k_1.predict(X_train_1) == y_train_1).mean()
print("TRAINING ACCURACY FOR N = 1:", KNN_model_k_1_accuracy)

TRAINING ACCURACY: 1.0


In [14]:
# prognozowanie na danych walidaycjnych
predictions_array = confusion_matrix(KNN_model_k_1.predict(x_valid_1), y_valid_1)
print(predictions_array)
# Znajdowanie klasy o największym prawdopodobieństwie wystąpienia w prognozie 
max_prawd = np.argmax(predictions_array[0]))
print(f'Najczęsciej wystepującym ubraniem jest: {class_clothes[max_prawd]}')


[[802   2  16  35   8   2 172   0   7   0]
 [  3 922   0   4   1   0   1   0   0   0]
 [ 26   2 833   8 117   0 128   0  10   1]
 [ 21  16   9 834  33   0  23   0   2   0]
 [  4   1 107  31 745   0  67   0   6   0]
 [  1   0   0   0   0 866   0   5   0   2]
 [128   5  98  45 105   3 625   0  14   0]
 [  0   0   0   0   0  79   0 971   1  29]
 [  3   0   1   2   2   4   4   0 908   1]
 [  0   0   1   0   0  48   0  63   3 984]]
0


In [19]:
max_prawd = np.argmax(predictions_array[0])
print(class_clothes[max_prawd])


T-shirt/top


In [15]:
# ocena dokładności prognozy na danych walidacyjnych 
KNN_model_k_1_accuracy = (KNN_model_k_1.predict(x_valid_1) == y_valid_1).mean()
print("VALIDATION ACCURACY for n = 1 :", KNN_model_k_1_accuracy)


TRAINING ACCURACY for n = 1 : 0.849


In [ ]:
# testowanie
KNN_model_k_1_test = KNN_model_k_1.predict_proba(x_test_1)

Gdy przyjmujemy jako ilość sąsiadów k = 5

In [7]:
start = time.clock()
KNN_model_k_5 = KNeighborsClassifier(n_neighbors=5).fit(X_train_1, y_train_1)
macierz_5 = confusion_matrix(KNN_model_k_5.predict(X_train_1), y_train_1)

print(macierz_5)
print(np.argmax(macierz_5[0]))
print(class_clothes[np.argmax(macierz_5[0])])


[[3714   12   50  139   17    1  544    0   19    0]
 [   2 3951    2   12    3    0    4    0    2    0]
 [  36   22 3374   33  304    2  410    0   54    0]
 [  53   48   25 3650   87    0   63    0   14    0]
 [  11    3  281  118 3310    1  236    0   18    0]
 [   0    0    0    0    0 3437    0    6    3    7]
 [ 198   14  178   62  244   10 2734    0   34    1]
 [   2    0    1    0    0  307    0 3849   18   95]
 [  26    0    5    2    6    6   21    1 3888    0]
 [   0    1    0    0    0  212    1  146    5 3855]]
0
T-shirt/top


In [8]:
KNN_model_k_5_accuracy = (KNN_model_k_5.predict(X_train_1) == y_train_1).mean()
print("TRAINING ACCURACY FOR N = 5:", KNN_model_k_5_accuracy)


TRAINING ACCURACY FOR N = 5: 0.89405


In [10]:

KNN_model_k_5_accuracy = (KNN_model_k_5.predict(x_valid_1) == y_valid_1).mean()
confusion_matrix(KNN_model_k_5.predict(x_valid_1), y_valid_1)
print("VALIDATION ACCURACY FOR n = 5:", KNN_model_k_5_accuracy)
end = time.clock()
total = end - start
print(f'time:{total/60}' )

VALIDATION ACCURACY FOR n = 5: 0.8508
time:127.50950461666666


In [ ]:
# testowanie zbiorem testowym
KNN_model_k_5_test = KNN_model_k_5.predict_proba(x_test_1)

Gdy przyjmujemyjako ilość sądiadów k = 9 

In [7]:
# rozpoczęcie liczenia czasu
start = time.clock()
# dopasowanie modalu kNN z k = 9  dla danych 
KNN_model_k_9 = KNeighborsClassifier(n_neighbors=9).fit(X_train_1, y_train_1)
macierz_9 = confusion_matrix(KNN_model_k_9.predict(X_train_1) == y_train_1).mean()
print(macierz_9)
print(np.argmax(macierz_9[0]))
print(f'Najczęsciej występującym ubraniem jest: {class_clothes[np.argmax(macierz_9[0])]}')
KNN_model_k_9_accuracy = (KNN_model_k_9.predict(X_train_1) == y_train_1).mean()
print("TRAINING ACCURACY FOR N = 5:", KNN_model_k_5_accuracy)
# prognozowanie na danych walidacyjnych
KNN_model_k_9_accuracy = (KNN_model_k_9.predict(X_train_1) == y_train_1).mean()
print(confusion_matrix(KNN_model_k_9.predict(x_valid_1), y_valid_1))
print("VALIDATTION ACCURACY for n = 9 :", KNN_model_k_9_accuracy)
end = time.clock()
# zakończenie liczenia czasu
total = end - start
print(f'time:{total/60}' )

TRAINING ACCURACY: 0.873575
time:70.40844191666666


In [ ]:
# testowanie zbiorem testowym
KNN_model_k_9_test = KNN_model_k_9.predict_proba(x_test_1)

Fitting Gradient Boosting - metoda wykorzystująca  boosting 